In [1]:
!pip install optuna lightgbm imbalanced-learn

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.5 MB ? eta -:--:--
   --------------------- ------------------ 0.8/1.5 MB 1.6 MB/s eta 0:00:01
   ------------------------------------ --- 1.3/1.5 MB 2.0 MB/s eta 0:00:01
   ------------------------------------ --- 1.3/1.5 MB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 1.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:

import dagshub
dagshub.init(repo_owner='Vaibha3246', repo_name='influence_mirror', mlflow=True)

import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("https://dagshub.com/Vaibha3246/influence_mirror.mlflow")

Accessing as Vaibha3246

Initialized MLflow to track repo "Vaibha3246/influence_mirror"

Repository Vaibha3246/influence_mirror initialized!

In [3]:
# set or create an experiment
mlflow.set_experiment("exp_5 ml_algo_with_hp_tunning")


<Experiment: artifact_location='mlflow-artifacts:/adb82bab710d416190b0fea77cabca06', creation_time=1760159677050, experiment_id='3', last_update_time=1760159677050, lifecycle_stage='active', name='exp_5 ml_algo_with_hp_tunning', tags={}>

In [6]:
# ✅ Step 1: Imports
import optuna
import mlflow
import mlflow.lightgbm
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.sparse as sp

from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [7]:
import pandas as pd
df=pd.read_csv('preprocessing.csv').dropna(subset=['text_clean'])

In [8]:



# ✅ Step 2: Prepare Target
df['sentiment_numeric'] = df['sentiment_numeric'].map({-1: 2, 0: 0, 1: 1})
df = df.dropna(subset=['sentiment_numeric'])


# ✅ Step 3: Select Features
numeric_cols = [col for col in df.columns if col not in [
    'video_id', 'category', 'text', 'text_clean', 'sentiment',
    'dominant_emotion', 'published_at', 'sentiment_numeric'
]]

X_numeric = df[numeric_cols]
y = df['sentiment_numeric']

# Scale numeric features
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_numeric)

# Split train/test
X_train_num, X_test_num, y_train, y_test, train_idx, test_idx = train_test_split(
    X_numeric_scaled, y, df.index, test_size=0.2, random_state=42, stratify=y
)

# ✅ Step 4: TF-IDF Vectorization
tfidf = TfidfVectorizer(ngram_range=(1, 3), max_features=10000)
X_train_text = tfidf.fit_transform(df.loc[train_idx, 'text_clean'])
X_test_text = tfidf.transform(df.loc[test_idx, 'text_clean'])

# Combine text + numeric
X_train = sp.hstack([X_train_text, sp.csr_matrix(X_train_num)])
X_test = sp.hstack([X_test_text, sp.csr_matrix(X_test_num)])

# Handle class imbalance
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)


# ✅ Step 5: Define Optuna Objective for LightGBM
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 400),
        "max_depth": trial.suggest_int("max_depth", -1, 15),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 20, 150),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.5, 2.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 50),
        "objective": "multiclass",
        "num_class": len(np.unique(y)),
        "random_state": 42,
        "n_jobs": -1
    }

    model = LGBMClassifier(**params)
    model.fit(X_train_res, y_train_res)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    return acc


# ✅ Step 6: Run Optuna Tuning
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=25)

print("✅ Best LightGBM Params:", study.best_params)
print("✅ Best LightGBM Accuracy:", study.best_value)


# ✅ Step 7: Train and Log the Best Model
best_params = study.best_params
best_model = LGBMClassifier(**best_params)
best_model.fit(X_train_res, y_train_res)
y_pred = best_model.predict(X_test)

with mlflow.start_run(run_name="LightGBM_TFIDF_NUMERIC_SMOTE_Optuna_Best"):
    mlflow.set_tag("experiment_type", "algorithm_comparison")
    mlflow.set_tag("model_type", "LightGBM")

    # Log parameters
    mlflow.log_params(best_params)
    mlflow.log_param("vectorizer_type", "TF-IDF")
    mlflow.log_param("ngram_range", (1, 3))
    mlflow.log_param("max_features", 10000)
    mlflow.log_param("imbalance_method", "SMOTE")

    # Log metrics
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)

    classification_rep = classification_report(y_test, y_pred, output_dict=True)
    for label, metrics in classification_rep.items():
        if isinstance(metrics, dict):
            for metric, value in metrics.items():
                mlflow.log_metric(f"{label}_{metric}", value)

    # Confusion Matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix: LightGBM Best Model")
    plt.savefig("confusion_matrix_lightgbm_best.png")
    mlflow.log_artifact("confusion_matrix_lightgbm_best.png")
    plt.close()

    # Log the trained model
    mlflow.lightgbm.log_model(best_model, "best_lightgbm_model")

print("🎯 MLflow run logged successfully for LightGBM ✅")


[I 2025-10-11 19:53:46,726] A new study created in memory with name: no-name-6df1e621-910e-473a-8ed3-8037e3d271c6


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.662150 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 171641
[LightGBM] [Info] Number of data points in the train set: 73119, number of used features: 5011
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further

e:\influence-mirror\myenv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-10-11 19:54:58,862] Trial 0 finished with value: 0.7022823671985458 and parameters: {'n_estimators': 137, 'max_depth': 6, 'learning_rate': 0.022814824069783437, 'num_leaves': 41, 'subsample': 0.8456768727229932, 'colsample_bytree': 0.8444330343645524, 'reg_alpha': 0.03671011726652451, 'reg_lambda': 0.6576538438359767, 'min_child_samples': 20}. Best is trial 0 with value: 0.7022823671985458.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.459939 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 170418
[LightGBM] [Info] Number of data points in the train set: 73119, number of used features: 4855
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning]

e:\influence-mirror\myenv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-10-11 19:55:15,806] Trial 1 finished with value: 0.671884467784286 and parameters: {'n_estimators': 146, 'max_depth': 2, 'learning_rate': 0.049274418284796256, 'num_leaves': 77, 'subsample': 0.6201887321241413, 'colsample_bytree': 0.7450973303603325, 'reg_alpha': 0.929791507385514, 'reg_lambda': 0.7417374730218211, 'min_child_samples': 21}. Best is trial 0 with value: 0.7022823671985458.


...

✅ Best LightGBM Params: {'n_estimators': 364, 'max_depth': 0, 'learning_rate': 0.01581664713975589, 'num_leaves': 54, 'subsample': 0.6739659735137798, 'colsample_bytree': 0.6945404003108304, 'reg_alpha': 0.46561641448856517, 'reg_lambda': 1.9412438489923112, 'min_child_samples': 40}
✅ Best LightGBM Accuracy: 0.7381337103615431


e:\influence-mirror\myenv\Lib\site-packages\joblib\externals\loky\backend\context.py:131: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "e:\influence-mirror\myenv\Lib\site-packages\joblib\externals\loky\backend\context.py", line 247, in _count_physical_cores
    cpu_count_physical = _count_physical_cores_win32()
  File "e:\influence-mirror\myenv\Lib\site-packages\joblib\externals\loky\backend\context.py", line 299, in _count_physical_cores_win32
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "C:\Users\vaibh\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 554, in run
    with Popen(*popenargs, **kwargs) as proce

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.592405 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 148953
[LightGBM] [Info] Number of data points in the train set: 73119, number of used features: 2843
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


e:\influence-mirror\myenv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
2025/10/11 20:29:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LightGBM_TFIDF_NUMERIC_SMOTE_Optuna_Best at: https://dagshub.com/Vaibha3246/influence_mirror.mlflow/#/experiments/3/runs/9b31fedd433344b285b4d4632729cec4
🧪 View experiment at: https://dagshub.com/Vaibha3246/influence_mirror.mlflow/#/experiments/3
🎯 MLflow run logged successfully for LightGBM ✅
